# Análise de Ajuste de Threshold para Dados Desbalanceados

Este notebook demonstra como ajustar o threshold de classificação para melhorar o desempenho de modelos com dados desbalanceados, especialmente em termos de recall.

Os modelos de classificação padrão geralmente usam um threshold de 0.5 para decidir se um exemplo pertence à classe positiva ou negativa. No entanto, quando os dados estão desbalanceados (ou seja, uma classe é muito mais frequente que a outra), esse threshold padrão pode não ser ideal.

Neste notebook, vamos:
1. Carregar dados de candidatos com classes desbalanceadas
2. Treinar um modelo de classificação 
3. Avaliar o desempenho com threshold padrão (0.5)
4. Encontrar um threshold ótimo usando curvas ROC e Precision-Recall
5. Avaliar o desempenho com threshold ajustado
6. Visualizar o impacto de diferentes thresholds nas métricas

In [ ]:
# Importar bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
from sklearn.metrics import (confusion_matrix, classification_report, 
                            precision_recall_curve, roc_curve, auc,
                            f1_score, precision_score, recall_score, 
                            accuracy_score, average_precision_score, roc_auc_score)

# Configurações para visualização
plt.style.use('ggplot')
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Carregamento e Preparação dos Dados

Primeiro, vamos carregar os dados processados e verificar a distribuição das classes.
O nosso modelo está tentando prever a variável `target_sucesso` que indica se um candidato foi contratado com sucesso (1) ou não (0).

In [ ]:
# Carregar dados processados
try:
    df = pd.read_csv('../data/processed/complete_processed_data.csv')
    print(f"✅ Dados carregados: {df.shape[0]} registros, {df.shape[1]} colunas")
except FileNotFoundError:
    print("❌ Arquivo de dados não encontrado. Verifique o caminho.")
    
# Verificar a presença da variável target
target = 'target_sucesso'
if target in df.columns:
    # Distribuição da variável target
    target_dist = df[target].value_counts(normalize=True) * 100
    print("\n📊 Distribuição do Target:")
    for value, pct in target_dist.items():
        print(f"  - Classe {value}: {pct:.1f}%")
    
    # Visualizar distribuição
    plt.figure(figsize=(10, 6))
    ax = sns.countplot(x=target, data=df)
    plt.title('Distribuição da Variável Target (Sucesso na Contratação)')
    plt.xlabel('Sucesso na Contratação (0=Não, 1=Sim)')
    plt.ylabel('Número de Candidatos')
    
    # Adicionar valores nas barras
    for p in ax.patches:
        ax.annotate(f'{p.get_height()}', 
                    (p.get_x() + p.get_width()/2., p.get_height()), 
                    ha='center', va='bottom')
    
    plt.show()
else:
    print(f"❌ Variável target '{target}' não encontrada nos dados.")
    
# Separar features e target
if target in df.columns:
    X = df.drop(columns=[target])
    y = df[target]
    print(f"\n✅ Features e target separados: {X.shape[1]} features, {len(y)} exemplos")

## Carregamento do Modelo Treinado

Vamos carregar o modelo pré-treinado para avaliar seu desempenho com diferentes thresholds.
Primeiro, precisamos verificar se o modelo já existe.

In [ ]:
# Carregar o modelo treinado
model_path = '../models/scoring_model.pkl'

try:
    with open(model_path, 'rb') as file:
        model = pickle.load(file)
    print(f"✅ Modelo carregado com sucesso: {type(model).__name__}")
    
    # Extrair informações sobre o modelo
    if hasattr(model, 'steps'):
        for name, step in model.named_steps.items():
            print(f"  - {name}: {type(step).__name__}")
    elif hasattr(model, 'estimators_'):
        print(f"  - Número de estimadores: {len(model.estimators_)}")
    else:
        print("  - Informações detalhadas não disponíveis para este tipo de modelo")
except FileNotFoundError:
    print(f"❌ Modelo não encontrado em {model_path}")
    print("🔄 Treinando um modelo simples para demonstração...")
    
    # Se não encontrarmos o modelo, treinaremos um modelo simples para demonstração
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    
    # Dividir os dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Treinar um modelo RandomForest
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    print("✅ Modelo de demonstração treinado com sucesso!")

## Avaliação com Threshold Padrão (0.5)

Vamos avaliar o desempenho do modelo usando o threshold padrão de 0.5 para classificação e calcular diversas métricas para entender como o modelo se comporta com dados desbalanceados.

In [ ]:
# Dividir dados em treino e teste se não foi feito anteriormente
try:
    X_test
except NameError:
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"✅ Dados divididos em treino e teste: {len(X_train)} exemplos de treino, {len(X_test)} exemplos de teste")

# Obter probabilidades de predição
if hasattr(model, 'predict_proba'):
    y_proba = model.predict_proba(X_test)[:, 1]  # Probabilidade da classe positiva
    print("✅ Probabilidades de predição obtidas")
else:
    # Se o modelo não tiver predict_proba, usar decisão como probabilidade
    y_proba = model.predict(X_test)
    print("⚠️ Modelo não possui método predict_proba, usando decisões como probabilidades")

# Predições com threshold padrão de 0.5
threshold_default = 0.5
y_pred_default = (y_proba >= threshold_default).astype(int)

# Calcular métricas de classificação com threshold padrão
accuracy_default = accuracy_score(y_test, y_pred_default)
precision_default = precision_score(y_test, y_pred_default)
recall_default = recall_score(y_test, y_pred_default)
f1_default = f1_score(y_test, y_pred_default)
auc_default = roc_auc_score(y_test, y_proba)

# Exibir resultados
print("\n📊 Performance com Threshold Padrão (0.5):")
print(f"  - Acurácia: {accuracy_default:.4f}")
print(f"  - Precisão: {precision_default:.4f}")
print(f"  - Recall: {recall_default:.4f}")
print(f"  - F1-Score: {f1_default:.4f}")
print(f"  - AUC-ROC: {auc_default:.4f}")

# Exibir relatório de classificação detalhado
print("\n📋 Relatório de Classificação Detalhado:")
print(classification_report(y_test, y_pred_default))

# Visualizar matriz de confusão
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred_default)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Não Contratado', 'Contratado'],
            yticklabels=['Não Contratado', 'Contratado'])
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title(f'Matriz de Confusão (Threshold = {threshold_default})')
plt.show()

## Encontrando o Threshold Ótimo

Para dados desbalanceados, o threshold padrão de 0.5 geralmente não é ideal. Vamos usar diferentes métodos para encontrar um threshold mais adequado:

1. **Curva Precision-Recall**: Para datasets desbalanceados, esta curva é geralmente mais informativa que a curva ROC.
2. **Curva ROC**: Permite visualizar trade-off entre taxa de verdadeiros positivos e falsos positivos.
3. **F1-Score para diferentes thresholds**: Encontrar o threshold que maximiza o F1-Score.

Vamos implementar essas abordagens:

In [ ]:
# 1. Curva Precision-Recall
precision, recall, thresholds_pr = precision_recall_curve(y_test, y_proba)

# Calcular F1 score para cada threshold
f1_scores = []
for p, r in zip(precision, recall):
    if p + r == 0:  # Evitar divisão por zero
        f1 = 0
    else:
        f1 = 2 * (p * r) / (p + r)
    f1_scores.append(f1)

# Encontrar o threshold que maximiza o F1-score
optimal_idx_f1 = np.argmax(f1_scores)
optimal_threshold_f1 = thresholds_pr[optimal_idx_f1] if optimal_idx_f1 < len(thresholds_pr) else 0.5

# 2. Curva ROC
fpr, tpr, thresholds_roc = roc_curve(y_test, y_proba)

# Encontrar o threshold que maximiza a distância à linha aleatória (ponto mais distante da linha diagonal)
# Também conhecido como ponto J de Youden
distances = tpr - fpr
optimal_idx_roc = np.argmax(distances)
optimal_threshold_roc = thresholds_roc[optimal_idx_roc]

# 3. Calcular métricas para vários thresholds
thresholds_range = np.linspace(0.05, 0.95, 19)  # De 0.05 a 0.95 em 19 steps
metrics = []

for threshold in thresholds_range:
    y_pred = (y_proba >= threshold).astype(int)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    metrics.append({
        'threshold': threshold,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    })

metrics_df = pd.DataFrame(metrics)

# Encontrar o threshold que maximiza o F1-score na nossa grade
optimal_threshold_grid = metrics_df.loc[metrics_df['f1'].idxmax(), 'threshold']

print(f"Threshold ótimo (maximizando F1 na curva PR): {optimal_threshold_f1:.4f}")
print(f"Threshold ótimo (maximizando J de Youden na curva ROC): {optimal_threshold_roc:.4f}")
print(f"Threshold ótimo (maximizando F1 em nossa grade): {optimal_threshold_grid:.4f}")

# Escolher o threshold final (média dos métodos)
final_threshold = np.mean([optimal_threshold_f1, optimal_threshold_roc, optimal_threshold_grid])
print(f"\n🎯 Threshold recomendado final: {final_threshold:.4f}")

# Visualizar as curvas
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Curva Precision-Recall
ax1.plot(recall, precision, label='Curva Precision-Recall')
ax1.scatter(recall[optimal_idx_f1], precision[optimal_idx_f1], 
           color='red', label=f'Threshold Ótimo: {optimal_threshold_f1:.4f}')
ax1.set_xlabel('Recall')
ax1.set_ylabel('Precision')
ax1.set_title('Curva Precision-Recall')
ax1.legend()
ax1.grid(True)

# Curva ROC
ax2.plot(fpr, tpr, label=f'Curva ROC (AUC = {auc_default:.4f})')
ax2.plot([0, 1], [0, 1], 'k--', label='Aleatório')
ax2.scatter(fpr[optimal_idx_roc], tpr[optimal_idx_roc], 
           color='red', label=f'Threshold Ótimo: {optimal_threshold_roc:.4f}')
ax2.set_xlabel('Taxa de Falsos Positivos')
ax2.set_ylabel('Taxa de Verdadeiros Positivos')
ax2.set_title('Curva ROC')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## Visualização do Impacto de Diferentes Thresholds

Agora vamos visualizar como as métricas de desempenho (precisão, recall, F1 e acurácia) variam conforme mudamos o threshold de classificação.

In [ ]:
# Plotar as métricas para diferentes thresholds
plt.figure(figsize=(12, 8))

# Plotar cada métrica
plt.plot(metrics_df['threshold'], metrics_df['accuracy'], 'b-', label='Acurácia')
plt.plot(metrics_df['threshold'], metrics_df['precision'], 'g-', label='Precisão')
plt.plot(metrics_df['threshold'], metrics_df['recall'], 'r-', label='Recall')
plt.plot(metrics_df['threshold'], metrics_df['f1'], 'purple', label='F1-Score')

# Adicionar linhas para os thresholds ótimos
plt.axvline(x=optimal_threshold_f1, color='red', linestyle='--', alpha=0.5, label=f'Threshold PR: {optimal_threshold_f1:.4f}')
plt.axvline(x=optimal_threshold_roc, color='green', linestyle='--', alpha=0.5, label=f'Threshold ROC: {optimal_threshold_roc:.4f}')
plt.axvline(x=final_threshold, color='black', linestyle='-', alpha=0.7, label=f'Threshold Final: {final_threshold:.4f}')
plt.axvline(x=0.5, color='gray', linestyle=':', alpha=0.7, label='Threshold Padrão: 0.5')

# Configurar o gráfico
plt.title('Impacto do Threshold nas Métricas de Classificação')
plt.xlabel('Threshold')
plt.ylabel('Valor da Métrica')
plt.legend(loc='best')
plt.grid(True)
plt.xticks(np.arange(0, 1.05, 0.05))
plt.xlim([0, 1])
plt.ylim([0, 1])

plt.show()

# Visualização em formato de tabela para thresholds selecionados
selected_thresholds = [0.1, 0.2, 0.25, 0.3, 0.4, 0.5]
selected_metrics = []

for threshold in selected_thresholds:
    y_pred = (y_proba >= threshold).astype(int)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Contar positivos preditos
    positivos_preditos = np.sum(y_pred == 1)
    pct_positivos = (positivos_preditos / len(y_pred)) * 100
    
    selected_metrics.append({
        'threshold': threshold,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'positivos_preditos': positivos_preditos,
        'pct_positivos': pct_positivos
    })

# Converter para DataFrame e exibir
selected_df = pd.DataFrame(selected_metrics)
selected_df = selected_df.round(4)
selected_df

## Avaliação com Threshold Ajustado

Agora vamos avaliar o desempenho do modelo usando o threshold otimizado que encontramos. Vamos comparar as métricas e a matriz de confusão com o threshold padrão de 0.5.

In [ ]:
# Usar o threshold otimizado
threshold_optimized = final_threshold
y_pred_optimized = (y_proba >= threshold_optimized).astype(int)

# Calcular métricas de classificação com threshold otimizado
accuracy_optimized = accuracy_score(y_test, y_pred_optimized)
precision_optimized = precision_score(y_test, y_pred_optimized)
recall_optimized = recall_score(y_test, y_pred_optimized)
f1_optimized = f1_score(y_test, y_pred_optimized)
auc_optimized = auc_default  # AUC é invariante ao threshold

# Comparar resultados
results = {
    'Métrica': ['Acurácia', 'Precisão', 'Recall', 'F1-Score', 'AUC-ROC'],
    'Threshold Padrão (0.5)': [accuracy_default, precision_default, recall_default, f1_default, auc_default],
    f'Threshold Otimizado ({threshold_optimized:.3f})': [accuracy_optimized, precision_optimized, recall_optimized, f1_optimized, auc_optimized],
    'Diferença': [
        accuracy_optimized - accuracy_default,
        precision_optimized - precision_default,
        recall_optimized - recall_default,
        f1_optimized - f1_default,
        0  # AUC é invariante ao threshold
    ]
}

# Criar DataFrame para visualização
results_df = pd.DataFrame(results)
results_df = results_df.round(4)
results_df

# Calcular variação percentual nas métricas
percent_change = {
    'Métrica': ['Acurácia', 'Precisão', 'Recall', 'F1-Score'],
    'Variação %': [
        (accuracy_optimized - accuracy_default) / accuracy_default * 100,
        (precision_optimized - precision_default) / precision_default * 100,
        (recall_optimized - recall_default) / recall_default * 100 if recall_default > 0 else float('inf'),
        (f1_optimized - f1_default) / f1_default * 100 if f1_default > 0 else float('inf')
    ]
}
percent_df = pd.DataFrame(percent_change)
percent_df['Variação %'] = percent_df['Variação %'].round(2)

# Visualizar matrizes de confusão lado a lado
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Matriz de confusão com threshold padrão
cm_default = confusion_matrix(y_test, y_pred_default)
sns.heatmap(cm_default, annot=True, fmt='d', cmap='Blues', ax=ax1,
            xticklabels=['Não Contratado', 'Contratado'],
            yticklabels=['Não Contratado', 'Contratado'])
ax1.set_xlabel('Predito')
ax1.set_ylabel('Real')
ax1.set_title(f'Matriz de Confusão (Threshold = 0.5)')

# Matriz de confusão com threshold otimizado
cm_optimized = confusion_matrix(y_test, y_pred_optimized)
sns.heatmap(cm_optimized, annot=True, fmt='d', cmap='Blues', ax=ax2,
            xticklabels=['Não Contratado', 'Contratado'],
            yticklabels=['Não Contratado', 'Contratado'])
ax2.set_xlabel('Predito')
ax2.set_ylabel('Real')
ax2.set_title(f'Matriz de Confusão (Threshold = {threshold_optimized:.3f})')

plt.tight_layout()
plt.show()

# Exibir também a variação percentual
percent_df

## Conclusões e Recomendações

Com base nas análises realizadas, podemos tirar algumas conclusões importantes sobre o ajuste de threshold para o nosso modelo de scoring:

In [ ]:
# Criar função para salvar o threshold otimizado
def save_optimal_threshold(threshold, output_file='../config/optimal_threshold.txt'):
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    with open(output_file, 'w') as f:
        f.write(f"{threshold}")
    print(f"✅ Threshold otimizado salvo em {output_file}")

# Salvar o threshold otimizado em um arquivo de configuração
try:
    save_optimal_threshold(threshold_optimized)
except Exception as e:
    print(f"⚠️ Erro ao salvar o threshold: {str(e)}")

# Resumo das conclusões em formato de texto
print("📝 CONCLUSÕES DA ANÁLISE DE THRESHOLD:")
print("-" * 50)
print(f"1. O threshold padrão (0.5) resultou em:")
print(f"   - Alta precisão: {precision_default:.4f}")
print(f"   - Baixo recall: {recall_default:.4f}")
print(f"   - F1-score: {f1_default:.4f}")
print(f"   Este comportamento é típico em problemas com dados desbalanceados.")
print()
print(f"2. O threshold otimizado ({threshold_optimized:.4f}) resultou em:")
print(f"   - Precisão: {precision_optimized:.4f} ({(precision_optimized - precision_default) / precision_default * 100:.1f}%)")
print(f"   - Recall: {recall_optimized:.4f} ({(recall_optimized - recall_default) / recall_default * 100:.1f}%)" if recall_default > 0 else f"   - Recall: {recall_optimized:.4f} (∞%)")
print(f"   - F1-score: {f1_optimized:.4f} ({(f1_optimized - f1_default) / f1_default * 100:.1f}%)" if f1_default > 0 else f"   - F1-score: {f1_optimized:.4f} (∞%)")
print()
print("3. Recomendações:")
print(f"   - Usar threshold de {threshold_optimized:.4f} para melhorar o equilíbrio entre precisão e recall")
print("   - Configurar via variável de ambiente CLASSIFICATION_THRESHOLD")
print("   - Monitorar o impacto no ambiente de produção")
print("   - Reavaliar periodicamente conforme novos dados são coletados")
print("-" * 50)

## Como Implementar o Threshold Ajustado na API

O threshold otimizado pode ser implementado na API de scoring através das seguintes etapas:

1. **Configurar via variável de ambiente**: Definir `CLASSIFICATION_THRESHOLD=0.25` (ou o valor otimizado encontrado)

2. **Modificar o código de predição**: Atualizar a função de predição para usar o threshold configurável:

```python
def get_classification_threshold():
    """Obtém o threshold configurado ou usa o valor padrão otimizado"""
    try:
        threshold = float(os.environ.get("CLASSIFICATION_THRESHOLD", "0.25"))
        threshold = max(0.01, min(threshold, 0.99))  # Limitar entre 0.01 e 0.99
        return threshold
    except (ValueError, TypeError):
        return 0.25  # Valor otimizado padrão
        
def predict(data):
    # Obter probabilidade
    probability = model.predict_proba(data)[:, 1]
    
    # Usar threshold ajustado
    threshold = get_classification_threshold()
    prediction = (probability >= threshold).astype(int)
    
    return {"prediction": prediction, "probability": probability, "threshold": threshold}
```

3. **Documentar a mudança**: Explicar o motivo e o impacto da alteração para futuras referências

4. **Monitorar o desempenho**: Verificar o impacto do novo threshold no ambiente de produção